In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.visualization
import named_arrays as na
import optika
import esis

%matplotlib notebook

In [ ]:


grid = optika.vectors.ObjectVectorArray(
    wavelength=na.linspace(-1, 1, axis="wavelength", num=2) / 2,
    field=0.99 * na.Cartesian2dVectorLinearSpace(
        start=-1,
        stop=1,
        axis=na.Cartesian2dVectorArray("field_x", "field_y"),
        num=3,
    ),
    pupil=na.Cartesian2dVectorLinearSpace(
        start=-1,
        stop=1,
        axis=na.Cartesian2dVectorArray("pupil_x", "pupil_y"),
        num=25,
    ),
)

model = esis.flights.f2.optics.models.design_proposed_single(
    grid=grid,
    num_distribution=0,

)
model.system.transformation

In [ ]:
# fig, ax = plt.subplots(
#     figsize=(6, 6.5),
#     constrained_layout=True
# )
# ax.set_aspect("equal")
# model.system.plot(
#     components=('x','y'),
#     color="black",
#     kwargs_rays=dict(
#         color=na.ScalarArray(np.array(["tab:orange", "tab:blue"]), axes="wavelength"),
#         label=model.system.grid_input.wavelength.astype(int),
#     ),
# );
# handles, labels = ax.get_legend_handles_labels()
# labels = dict(zip(labels, handles))
# fig.legend(labels.values(), labels.keys());

In [ ]:
fig, ax = plt.subplots()

detector = model.system.surfaces[-1]
detector_plot = detector.plot(components=('x', 'y'), ax=ax, transform=None)

rays = model.system.rayfunction_default.outputs
rays = rays[rays.unvignetted]
rays = detector.transformation.inverse(rays)
na.plt.scatter(rays.position.x,
               rays.position.y)
ax.set_aspect("equal")

fig, ax = plt.subplots()

detector = model.system.surfaces_all[-1]
detector_plot = detector.plot(
    components=('x', 'y'),
    ax=ax,
    transformation=detector.transformation.inverse,
    # transformation=None,
)

rays = model.system.rayfunction_default.outputs
rays = detector.transformation.inverse(rays)
na.plt.scatter(
    rays.position.x,
    rays.position.y,
    where=rays.unvignetted,
)


In [ ]:


# fig, ax = plt.subplots(
#     figsize=(6, 6.5),
#     constrained_layout=True
# )
# ax.set_aspect("equal")
# model.system.plot(
#     components=("x", "y"),
#     color="black",
#     kwargs_rays=dict(
#         color=na.ScalarArray(np.array(["tab:orange", "tab:blue"]), axes="wavelength"),
#         label=model.system.grid_input.wavelength.astype(int),
#     ),
# )

In [ ]:

object_distance = np.sqrt(
    np.square(model.grating.translation.z - model.field_stop.translation.z) + np.square(model.grating.distance_radial))
image_distance = np.sqrt(np.square(model.grating.translation.z - model.detector.translation.z) +
                         np.square(model.grating.distance_radial - model.detector.distance_radial)
                         )
magnification = image_distance / object_distance
magnification


In [ ]:
print(model.primary_mirror.translation.z)
print(model.field_stop.translation.z)
print(model.grating.translation.z)
print(model.detector.translation.z)

In [ ]:
hole_spacing = 4 * 25.4 * u.mm
hole_spacing.to(u.mm)

In [ ]:
new_primary_f = model.primary_mirror.sag.focal_length + hole_spacing * 6
new_primary_f


In [ ]:
new_fs_z = -new_primary_f

In [ ]:
extension_ratio = new_primary_f / (1000 * u.mm)
extension_ratio

In [ ]:

old_grating_z = model.grating.translation.z - model.field_stop.translation.z
new_grating_z = old_grating_z * extension_ratio + new_fs_z
new_grating_z

In [ ]:
model.primary_mirror.sag.focal_length = new_primary_f
model.field_stop.translation.z = new_fs_z
model.grating.translation.z = new_grating_z + 30 * u.mm

In [ ]:
object_distance = np.sqrt(
    np.square(model.grating.translation.z - model.field_stop.translation.z) + np.square(model.grating.distance_radial))
image_distance = np.sqrt(np.square(model.grating.translation.z - model.detector.translation.z) +
                         np.square(model.grating.distance_radial - model.detector.distance_radial)
                         )
magnification = image_distance / object_distance
magnification
print(object_distance)
print(image_distance)

In [ ]:
new_grating_f = image_distance * object_distance / (image_distance + object_distance)
new_grating_f

In [ ]:
model.grating.sag.radius = 2 * new_grating_f.nominal
model.grating.yaw = -2.5 * u.deg

c0 = 1 / (1850 / u.mm)
c1 = -2.852e-5 * (u.um / u.mm)
c2 = -2.112e-7 * (u.um / u.mm ** 2)

model.grating.rulings.spacing.coefficients[0].nominal = c0
model.grating.rulings.spacing.coefficients[1].nominal = c1
model.grating.rulings.spacing.coefficients[2].nominal = c2


In [ ]:
print(model.primary_mirror.translation.z)
print(model.field_stop.transformation)
print(model.grating.translation.z)
print(model.detector.translation.z)

model.central_obscuration.translation.z = model.grating.translation.z - 25 * u.mm
model.front_aperture.translation.z = model.grating.translation.z - 100 * u.mm

In [ ]:
model.filter.radius_clear = 40 * u.mm

In [ ]:
model.detector.yaw

In [ ]:
del model.system

wavelength = na.ScalarArray([465, 499] * u.AA, axes='wavelength')
# rays = model.system.raytrace(wavelength=wavelength)
# rays.outputs.shape

In [ ]:
# fig, ax = plt.subplots(
#     figsize=(6, 6.5),
#     constrained_layout=True
# )
# ax.set_aspect("equal")
# model.system.plot(
#     components=("z", "x"),
#     color="black",
#     kwargs_rays=dict(
#         color=na.ScalarArray(np.array(["tab:orange", "tab:blue"]), axes="wavelength"),
#         label=model.system.grid_input.wavelength.astype(int),
#     ),
#     plot_rays_vignetted=False,
#     # wavelength=wavelength,
# )
# handles, labels = ax.get_legend_handles_labels()
# labels = dict(zip(labels, handles))
# fig.legend(labels.values(), labels.keys());

In [ ]:
fig, ax = plt.subplots()

detector = model.system.surfaces_all[-1]
detector_plot = detector.plot(
    components=('x', 'y'),
    ax=ax,
    transformation=detector.transformation.inverse,
    # transformation=None,
)

rays = model.system.raytrace(wavelength=wavelength).outputs[dict(surface=-1)]

rays = detector.transformation.inverse(rays)
na.plt.scatter(
    rays.position.x,
    rays.position.y,
    where=rays.unvignetted,
)


In [ ]:
x = rays.position.x
x = x - x.mean(axis=('pupil_x', 'pupil_y'))
y = rays.position.y
y = y - y.mean(axis=('pupil_x', 'pupil_y'))
rms_spot_size = np.sqrt(np.square(x) + np.square(y))
rms_spot_size.to(u.um)

In [ ]:
np.abs(rays.position[dict(wavelength=0)].x.mean()) - 8 * u.mm

In [ ]:
def esis_ii_merit(
        guesses,
        model=None,
        # wavelength = None, 
        weight=None,
        # units = None
):
    # guess = [guess*unit for guess,unit in zip(guesses,units)]
    print(guesses)
    grating_yaw, grating_c0, grating_c1, grating_c2, grating_roc, detector_yaw = guesses

    model.grating.yaw = grating_yaw * u.deg
    model.grating.rulings.spacing.coefficients[0] = grating_c0 * u.mm
    model.grating.rulings.spacing.coefficients[1] = grating_c1 * u.um / u.mm
    model.grating.rulings.spacing.coefficients[2] = grating_c2 * u.um / u.mm ** 2
    model.grating.sag.radius = grating_roc * u.mm
    model.detector.yaw = detector_yaw * u.deg

    wavelength = na.ScalarArray([465, 499] * u.AA, axes='wavelength')

    del model.system
    rays = model.system.raytrace(wavelength=wavelength).outputs[dict(surface=-1)]
    detector = model.system.surfaces_all[-1]

    rays = detector.transformation.inverse(rays)

    x = rays.position.x
    x = x - x.mean(axis=('pupil_x', 'pupil_y'))
    y = rays.position.y
    y = y - y.mean(axis=('pupil_x', 'pupil_y'))

    r_squared = np.square(x) + np.square(y)
    rms_spot_size = np.sqrt(r_squared)

    t1 = np.abs(rays.position[dict(wavelength=0)].x.mean()) - 7.35 * u.mm
    t2 = np.abs(rays.position[dict(wavelength=-1)].x.mean()) - 7.35 * u.mm

    t = rms_spot_size.mean().nominal
    merit = t + weight * np.sqrt(np.square(t1.nominal) + np.square(t2.nominal))

    print(t.value, t1.nominal.value, t2.nominal.value)
    print(merit.ndarray.value)
    return merit.ndarray.value





In [ ]:
model.grating.sag.radius

In [ ]:
from scipy.optimize import minimize, differential_evolution

guess = [
    model.grating.yaw,
    model.grating.rulings.spacing.coefficients[0].nominal,
    model.grating.rulings.spacing.coefficients[1].nominal,
    model.grating.rulings.spacing.coefficients[2].nominal,
    model.grating.sag.radius,
    model.detector.yaw
]
guess_units = [g.unit for g in guess]
print(guess_units)

guess_values = [g.value for g in guess]
guess_values = [-2.47016209e+00, 5.49599486e-04, -1.80866504e-05, -1.08341748e-07,
                9.17468102e+02, -1.04771264e+01]

print(guess_values)


In [ ]:


esis_ii_merit(guess_values, model=model, weight=1)

bounds = [(-2.6, -2.45), (1 / 1900, 1 / 1800), (-3e-5, -1e-5), (-2.15e-7, -1e-7), (917, 921), (-15, -10)]

fit = minimize(esis_ii_merit, guess_values, (model, 1),
               bounds=bounds,
               )
# fit = differential_evolution(esis_ii_merit,x0=guess_values,args = (model,1),bounds=bounds)


In [ ]:
guess = [-2.53962948e+00, 5.34862656e-04, -2.10590822e-05, -1.94958850e-07,
         9.18952281e+02, -1.22332163e+01]

guess = [-2.47016209e+00, 5.49599486e-04, -1.80866504e-05, -1.08341748e-07,
         9.17468102e+02, -1.04771264e+01]

# guess = [-2.53962948e+00,  0, 0, 0,
#   9.1895221e+02, 0]

# grating_yaw, grating_c0, grating_c1, grating_c2, grating_roc, detector_yaw = fit.x
grating_yaw, grating_c0, grating_c1, grating_c2, grating_roc, detector_yaw = guess

model.grating.yaw = grating_yaw * u.deg
model.grating.rulings.spacing.coefficients[0] = grating_c0 * u.mm
model.grating.rulings.spacing.coefficients[1] = grating_c1 * u.um / u.mm
model.grating.rulings.spacing.coefficients[2] = grating_c2 * u.um / u.mm ** 2
model.grating.sag.radius = grating_roc * u.mm
model.detector.yaw = detector_yaw * u.deg

In [ ]:
# model.grating.distance_radial = 21 * u.mm

In [ ]:
del model.system
fig, ax = plt.subplots(
    figsize=(6, 6.5),
    constrained_layout=True
)
ax.set_aspect("equal")
model.system.plot(
    components=("z", "x"),
    color="black",
    kwargs_rays=dict(
        color=na.ScalarArray(np.array(["tab:orange", "tab:blue"]), axes="wavelength"),
        label=model.system.grid_input.wavelength.astype(int),
    ),
    plot_rays_vignetted=False,
    # wavelength=wavelength,
)
handles, labels = ax.get_legend_handles_labels()
labels = dict(zip(labels, handles))
fig.legend(labels.values(), labels.keys());

In [ ]:
fig, ax = plt.subplots()

detector = model.system.surfaces_all[-1]
detector_plot = detector.plot(
    components=('x', 'y'),
    ax=ax,
    transformation=detector.transformation.inverse,
    # transformation=None,
)
del model.system
rays = model.system.raytrace(wavelength=wavelength).outputs[dict(surface=-1)]

rays = detector.transformation.inverse(rays)
na.plt.scatter(
    rays.position.x,
    rays.position.y,
    where=rays.unvignetted,
)